<div align="center"><h1>HPC / SENAI / Hackathon (2022.2)<br>
Brute Force Algorithm </h1></div>

Davi Machado Costa$^1$, João Felipe de Araújo Caldas$^1$

$^1$ Universidade SENAI CIMATEC, Salvador, Bahia, Brasil 

## Introdução

As técnicas de paralelismo compreendem em aplicação de estratégias baseadas na utilização de processamento paralelo manipulando diferentes recursos computacionais. Alguma dessas técnicas compreendem a utilização de bibliotecas paralelas como `OpenMP`, `MPI` e `CUDA`. Cada uma dessas bibliotecas consiste na manipulação de recursos computacionais diferentes, podendo ser utilizadas de forma híbrida, a fim da obtenção de máximo desempenho. No OpenMP e CUDA manipulamos *Threads*, enquanto no MPI, *Processos*, sendo de real relevância os impactos destas unidades de processamento frente aos recursos computacionais alocados. 

A seguir será apresentado um código sequencial para a quebra de senha de até 20 caracteres utilizando um algoritmo de *Força Bruta*. O objetivo básico será inserir técnicas de paralelismo ao código, tal que serão considerados alguns itens nas aplicações finais:

* `Análise dos Custos Computacionais das Aplicações Sequênciais e Paralelas`
    + Profilling CPU (gprof)
    + Profiling GPU (nsys)
* `Estudo das Estruturas Algorítmicas das Aplicações Paralelas`
    + Modelos Algorítmicos Aplicados
    + Características da inserção da API
* `Análise de Desempenho`
     + Experimentação de Parâmetros Ótimos (Melhores valores de Processos, Threads e Grid Computacional)
     + Indices de eficiência (Speedup)

## Regras do Hackathon

* Os idiomas oficiais desse HPC Hackathon são: inglês e português;

* Este ano a competição será em grupos de 2 ou 3 pessoas;

* Tópico Principal: Portabilidade e Otimização de Código;
    
* Os participantes disponibilizarão os resultados através um repositório git pessoal que será configurado pelos participantes e/ou pela ferramenta GOOGLE COLAB;

* Além do código modificado, a resolução deve conter scripts de execução automática para obter os Parâmetros Ótimos e os Speedups;

* O código produzido será avaliado em 2 pontos: desempenho e speedup;

* Os participantes devem codificar e comentar seus códigos;

* Os participantes acessarão o supercomputador via ssh com suas contas previamente configuradas;

* As habilidades necessárias são: Git, Google Colab, Jupyter Notebook, C, C++, Unix, shell, bash, OpenMP, CUDA, MPI;

Boa sorte e boa codificação!

## Código Sequencial 

In [ ]:
%%writefile bruteForce.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

time_t t1, t2;

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  for(j = 0; j < max; j++){
    if(j == pass_decimal){

      int index = 0;

      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';
        
      time (&t2);
      double dif;
      dif = difftime (t2, t1);

      printf("%1.0f\n", dif);
      break;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);

  time (&t1);
  bruteForce(password);

  return 0;
}

Overwriting bruteForce.c


In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3

In [ ]:
!./bruteForce _Hacka1

136


Passo a passo da execução do algoritmo de força bruta:


1.   Recebe por parâmetro uma senha a ser descoberta;
2.   Transforma a senha em binário para diminuir o custo computacional da busca; 
3.   Realiza a busca através de um laço de repetição que soma e verifica se o número binário é igual ao da senha;
4.   Retorna a senha encontrada em binário e *string*, juntamente ao tempo de execução.

## Análise dos Custos Computacionais da Aplicação Sequencial

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3 -pg

In [ ]:
!./bruteForce senhazz

Try to broke the password: senhazz
Found password!
Password in decimal base: 10446703561
Found password: senhazz

3.00 seconds


In [ ]:
!gprof -b bruteForce gmon.out

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
100.17      0.72     0.72                             bruteForce


"O comando `gprof` é útil para identificar como um programa consome recursos do processador. Para descobrir quais funções (rotinas) no programa estão usando o processador, você pode criar um perfil do programa com o comando `gprof`." (IBM)

Utilizou-se o comando `gprof` para analisar o custo computacional da aplicação sequencial e o resultado apontou que praticamente 100% do custo vem da função `bruteForce`, onde existe um laço de repetição que testa todas as possíveis combinações de caracteres até encontrar a senha.

Sendo assim, as técnicas de paralelização devem ser aplicadas nessa função a fim de diminuir o tempo de execução da aplicação, tornando-a mais eficiente.

---
## Estudo das Estruturas Algorítmicas das Aplicações Paralelas

### Multicore (OPENMP)

In [ ]:
%%writefile bruteForce-omp.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <omp.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

time_t t1, t2;

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  #pragma omp parallel for private(j)
  for(j = 0; j < max; j++){
    if(j == pass_decimal){

      int index = 0;


      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';

      time (&t2);
      double dif;
      dif = difftime (t2, t1);
      printf("%s\t%d\t%1.2f\n", s, omp_get_num_threads(), dif);

      exit(0);
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);

  time (&t1);
  bruteForce(password);

  return 0;
}

Overwriting bruteForce-omp.c


In [ ]:
!gcc bruteForce-omp.c -o bruteForce-omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=16 ./bruteForce-omp senhate

senhate	16	0.00


"A API OpenMP oferece suporte à programação paralela de memória compartilhada multiplataforma em C/C++ e Fortran. A API OpenMP define um modelo portátil e escalável com uma interface simples e flexível para desenvolver aplicativos paralelos em plataformas desde o *desktop* até o supercomputador." (OpenMP)

Ao tentar paralelizar o código com OpenMP, houve uma limitação referente ao uso da palavra reservada "break" dentro do laço de repetição, pois segundo a Microsoft, o OpenMP não disponibiliza um mecanismo para para *loops* paralelos. Por esse motivo, outras alternativas foram consideradas na tentativa de superar essa limitação, como por exemplo o uso de *flags* que, por meio de condições, deixa de buscar a senha após já tê-la encontrada. Porém, essa alternativa se mostrava menos eficiente que a sequencial, visto que as demais *threads* continuavam seu processamento mesmo após achar a senha. Por fim, a solução que se mostrou mais eficiência foi através da utilização do comando `exit(0)` que interrompe imediatamente a execução do programa e retorna um código de sucesso logo após encontrar a senha.

<h5>
  <center>
    <b>Figura 1</b> - Modelo de programação <i>multithreading</i>
  </center>
</h5>
<img src="https://www.baeldung.com/wp-content/uploads/sites/4/2020/07/multithreading.png" alt="Multithreading Programming">
<h5>
  <center>
    Fonte: Baeldung, 2022
  </center>
</h5>

A Figura 1 representa o funcionamento do código ao utilizar programação *multithreading*, em que a *thread* principal inicia as demais para que cada uma realize sua parte da iteração no laço de repetição até encontrar a senha e interromper a execução do programa. Ademais, não foi necessário trocar dados entre as *threads* trabalhadoras, já que existe apenas a variável privada "j".

### Multiprocessor (MPI)

In [ ]:
%%writefile bruteForce-mpi.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20
#define MASTER 0

time_t t1, t2;

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass, int base, int size, long long int min, long long int max, int numtasks) 
{
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  char s[MAXIMUM_PASSWORD];

  for(j = min; j < max; j++){
    if(j == pass_decimal){
     
      int index = 0;

  
      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';

      time (&t2);
      double dif;
      dif = difftime (t2, t1);
      printf("%s\t%d\t%1.2f\n", s, numtasks, dif);

      MPI_Abort(MPI_COMM_WORLD, MPI_SUCCESS);
    }
  }

}

int main(int argc, char **argv) 
{
  int numtasks, taskid;

  // Inicializa o MPI
  MPI_Init( &argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
  MPI_Status status;
  MPI_Comm_rank(MPI_COMM_WORLD, &taskid);

  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);

  // Calcula os intervalos que cada processo vai trabalhar
  int base = END_CHAR - START_CHAR + 2;
  int size = strlen(password);
  long long int max = my_pow(base, size);
  long long int partialMax = max / numtasks;
  long long int taskMin = taskid * partialMax;
  long long int taskMax = (taskid + 1) * partialMax;
  int rest = max % numtasks;
  
  time (&t1);
  if (rest && taskid == numtasks - 1) {
    // Se houver resto da divisão, acrescenta no intervalo do último processo
    bruteForce(password, base, size, taskMin, taskMax + rest, numtasks);
  } else {
    bruteForce(password, base, size, taskMin, taskMax, numtasks);
  }

  MPI_Finalize();
  return 0;
}

Overwriting bruteForce-mpi.c


In [ ]:
!mpicc bruteForce-mpi.c -o bruteForce-mpi -fopenmp -std=c99 -O3

In [ ]:
!mpirun -x MXM_LOG_LEVEL=error -np 32 --allow-run-as-root ./bruteForce-mpi senhate 2>/dev/null

senhate	32	0.00


"O Open MPI Project é uma implementação de Interface de Passagem de Mensagens de *software* livre desenvolvida e mantida por um consórcio de parceiros acadêmicos, de pesquisa e da indústria" (Open MPI). Esta interface permite a inicialização e a comunicação entre múltiplos processos do computador, possibilitando a divisão de tarefas para diminuir o custo computacional de uma aplicação. 

A Figura 2 mostra o modelo mestre-trabalhador utilizado para construir a solução com Open MPI, em que cada processo recebe um intervalo de busca e a senha a ser encontrada. Sendo assim, quando algum processo encontrar a senha, o MPI é abortado e o resultado é exibido em seguida.

<h5>
  <center>
    <b>Figura 2</b> - Modelo MPI para busca da senha
  </center>
</h5>
<img src="./mpi.jpeg" alt="Modelo mestre-trabalhador">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

### Multiprocessor + Multicore (MPI + OpenMP)

In [ ]:
%%writefile bruteForce-mpi+omp.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>
#include <omp.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20
#define MASTER 0

time_t t1, t2;
int numtasks;

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass, int base, int size, long long int min, long long int max) 
{
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  char s[MAXIMUM_PASSWORD];

  #pragma omp parallel for private(j)
  for(j = min; j < max; j++){
    if(j == pass_decimal){
   
      int index = 0;

      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';



      time (&t2);
      double dif;
      dif = difftime (t2, t1);
      printf("%s\tP%d/T%d\t%1.2f\n", s, numtasks, omp_get_num_threads(), dif, numtasks);

      MPI_Abort(MPI_COMM_WORLD, MPI_SUCCESS);
    }
  }

}

int main(int argc, char **argv) 
{
  int taskid;

  // Inicializa o MPI
  MPI_Init( &argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numtasks);
  MPI_Status status;
  MPI_Comm_rank(MPI_COMM_WORLD, &taskid);

  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);

  // Calcula os intervalos que cada processo vai trabalhar
  int base = END_CHAR - START_CHAR + 2;
  int size = strlen(password);
  long long int max = my_pow(base, size);
  long long int partialMax = max / numtasks;
  long long int taskMin = taskid * partialMax;
  long long int taskMax = (taskid + 1) * partialMax;
  int rest = max % numtasks;

  time (&t1);
  if (rest && taskid == numtasks - 1) {
    // Se houver resto da divisão, acrescenta no intervalo do último processo
    bruteForce(password, base, size, taskMin, taskMax + rest);
  } else {
    bruteForce(password, base, size, taskMin, taskMax);
  }

  MPI_Finalize();
  return 0;
}

Overwriting bruteForce-mpi+omp.c


In [ ]:
!mpicc bruteForce-mpi+omp.c -o bruteForce-mpi+omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=256 mpirun -np 8 --allow-run-as-root ./bruteForce-mpi+omp senha

--------------------------------------------------------------------------

  Local host:   login7
  Local device: mlx5_2
--------------------------------------------------------------------------
[1669324089.550494] [login7:188373:0]         shm.c:65   MXM  WARN  Could not open the KNEM device file at /dev/knem : No such file or directory. Won't use knem.
[1669324089.550932] [login7:188376:0]         shm.c:65   MXM  WARN  Could not open the KNEM device file at /dev/knem : No such file or directory. Won't use knem.
[1669324089.550883] [login7:188378:0]         shm.c:65   MXM  WARN  Could not open the KNEM device file at /dev/knem : No such file or directory. Won't use knem.
[1669324089.550501] [login7:188382:0]         shm.c:65   MXM  WARN  Could not open the KNEM device file at /dev/knem : No such file or directory. Won't use knem.
[1669324089.554536] [login7:188374:0]         shm.c:65   MXM  WARN  Could not open the KNEM device file at /dev/knem : No such file or directory. Won't use

A figura 3 representa a combinação entre MPI e OpenMP, tornando possível que em cada processo sejam executadas múltiplas *threads*. Ou seja, cada processo realiza a busca da senha em um intervalo específico utilizando múltiplas *threads* para parelelizar e tornar mais eficiente as iterações no laço de repetição.

<h5>
  <center>
    <b>Figura 3</b> - Modelo híbrido para busca da senha
  </center>
</h5>
<img src="./mpi+omp.jpeg" alt="Modelo mestre-trabalhador com multithreading">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

### GPU (CUDA)

In [ ]:
%%writefile bruteForce-cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <assert.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20
#define MAX_THREADS_PER_BLOCK 1024
#define BLOCKS_PER_SM 32

// Retorna erro, se houver, de uma função CUDA
inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

__device__ long long my_pow(long long x, int y) {
  long long res = 1;
  if (y == 0)
    return res;
  else
    return x * my_pow(x, y - 1);
}

// Calcula o tamanho de uma string
// Obs.: CUDA não tem suporte para a função strlen de string.h
__device__ int my_strlen(char *s) {
  int sum = 0;
  while (*s++) sum++;
  return sum;
}

__global__ void bruteForce(char * pass) {
  int pass_b26[MAXIMUM_PASSWORD];

  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = my_strlen(pass);;

  for (int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1;

  for (int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];
  // Calcula a iteração a partir da quantidade de threads vezes o id do bloco mais o id da thread
  long long int j = blockIdx.x * blockDim.x + threadIdx.x;

  // Realiza loop-stride para processar dados maiores que a quantidade de threads na GPU
  // Uma única thread processa mais de um dado
  while (j < max) {
    if (j == pass_decimal) {
   
      int index = 0;

      while (j > 0) {
        s[index++] = START_CHAR + j % base - 1;
        j /= base;
      }
      s[index] = '\0';

      break;
    }
    // Calcula o stride pela multiplicação entre a quantidade de blocos e threads 
    j += blockDim.x * gridDim.x;
  }
}

int main(int argc, char ** argv) {
  char *password;
  time_t t1, t2;
  double dif;

  // Aloca a senha para ser acessada tanto pela CPU quanto pela GPU
  checkCuda( cudaMallocManaged( & password, MAXIMUM_PASSWORD * sizeof(char)) );

  strcpy(password, argv[1]);

  int deviceId, numberOfSMs;
  checkCuda( cudaGetDevice( & deviceId) );
  // Pega a quantidade de SMs presentes na GPU
  checkCuda( cudaDeviceGetAttribute( & numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId) );
  // Multiplica a quantidade de SMs pela quantidade de blocos presentes em cada um
  int number_of_blocks = numberOfSMs * BLOCKS_PER_SM;
  int threads_per_block = MAX_THREADS_PER_BLOCK;


  time( & t1);
  bruteForce <<< number_of_blocks, threads_per_block >>> (password);
  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );
  time( & t2);

  dif = difftime(t2, t1);
  printf("%s\tB%d/T%d\t%1.2f\n", password, number_of_blocks, threads_per_block, dif);

  checkCuda( cudaFree(password) );

  return 0;
}

Overwriting bruteForce-cuda.cu


In [ ]:
!nvcc bruteForce-cuda.cu -o bruteForce-cuda

bruteForce-cuda.cu(60): warning #550-D: variable "s" was set but never used



In [ ]:
!./bruteForce-cuda _Hacka1

_Hacka1	B2560/T1024	13.00


"CUDA é uma plataforma de computação paralela e modelo de programação desenvolvido pela NVIDIA para computação geral em unidades de processamento gráfico (GPUs)" (NVIDIA). Desse modo, é importante compreender a arquitetura de uma GPU para entender como o CUDA atua.

<h5>
  <center>
    <b>Figura 4</b> - Perspectiva de <i>hardware</i> de uma GPU
  </center>
</h5>
<img src="https://www.researchgate.net/publication/333308943/figure/fig1/AS:762520998518785@1558810581846/Typical-NVIDIA-GPU-architecture-The-number-of-SMXs-and-the-computation-resources-inside.ppm">
<h5>
  <center>
    Fonte: Researchgate, Yehia Arafa
  </center>
</h5>

A Figura 4 representa a arquitetura de uma GPU, onde existem múltiplos *Streaming Multiprocessors* contendo diversos *cores*. 

Ao analisar Figura 5, é possível estabelecer uma relação entre o *software* (*grid*, *block* e *thread*) e o *hardware*, em que cada *thread* é executada por um núcleo (*core*), cada bloco é execudado por um SM e o grid é executado por toda a GPU através de múltiplos SMs.

Por padrão, as GPUs da NVIDIA atuais contém 32 blocos por SM, sendo que cada bloco pode ter no máximo 1024 *threads*.

<h5>
  <center>
    <b>Figura 5</b> - Relação <i>software</i> e <i>hardware</i> de uma GPU
  </center>
</h5>
<img src="https://pic1.zhimg.com/v2-701598e4e2f6a014d925005094fcdadc_b.jpg" alt="Sofware e hardware GPU">
<h5>
  <center>
    Fonte: The Beard, 2020
  </center>
</h5>

Sabendo disso, a lógica foi construída de forma a utilizar o máximo poder computacional da GPU. Para isso, O *kernel* foi executado com a maior quantidade de *threads* disponíveis na GPU, sendo que cada uma realiza sua busca através da técnica de *loop stride* representada pela Figura 6. 


<h5>
  <center>
    <b>Figura 6</b> - Lógica de <i>loop stride</i>
  </center>
</h5>
<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2021/03/grid-stride-1.png" alt="Sofware e hardware GPU">
<h5>
  <center>
    Fonte: NVIDIA, 2021
  </center>
</h5>

A Figura 6, representa uma técnica necessária para situações em que a quantidade de dados a serem processados é maior que a quantidade de *threads* disponíveis na GPU. Dessa forma, as *threads* precisam realizar mais de uma tarefa, possibilitando iterar todas as combinações possíveis de uma senha muito grande, por exemplo.

## Análise de Desempenho

In [ ]:
!nvidia-smi

Mon Nov 28 09:29:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:5E:00.0 Off |                    0 |
| N/A   28C    P0    24W / 250W |     70MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   

Esse script foi utilizado para transferir arquivos entre o computador pessoal e o OGBON, e vice-versa.

In [ ]:
%%bash
# OGBON -> PC
scp -i ~/.ssh/cimatec-ogbon-access-key.txt -P 5001 -r alu16@ogbon-login7.fieb.org.br:/home/alu16/davi/ .

# PC -> OGBON
scp -i ~/.ssh/cimatec-ogbon-access-key.txt -P 5001 -r /home/dmcdavi/Documents/HPC/ alu16@ogbon-login7.fieb.org.br:/home/alu16/

Esse script foi utilizado para conectar o computador pessoal ao OGBON através do protocolo SSH previamente configurado e autenticado, possibilitando rodar o presente notebook através do Jupyter na porta 8559. 

In [ ]:
%%bash
ssh ogbon-exerno -L 8559:*:8559

module load openmpi/4.1.1-cuda-11.6-ofed-5.4
module load anaconda3/2020.07

jupyter lab --port=8559

Esse script tem o propósito de compilar e permitir a execução todos os códigos escritos.

In [ ]:
%%bash
#!/bin/sh

gcc bruteForce.c -o bruteForce -std=c99 -O3
gcc bruteForce-omp.c -o bruteForce-omp -fopenmp -std=c99 -O3
nvcc -I/opt/share/openmpi/4.1.1-cuda/include -L/opt/share/openmpi/4.1.1-cuda/lib64 -DprintLabel -lnccl -lmpi -Xcompiler -fopenmp -o bruteForce-mpi bruteForce-mpi.c
mpicc bruteForce-mpi+omp.c -o bruteForce-mpi+omp -fopenmp -std=c99 -O3
nvcc bruteForce-cuda.cu -o bruteForce-cuda -O3

chmod +x bruteForce
chmod +x bruteForce-omp
chmod +x bruteForce-mpi
chmod +x bruteForce-mpi+omp
chmod +x bruteForce-cuda

bruteForce-cuda.cu(60): warning #550-D: variable "s" was set but never used



Esse script roda as soluções sequecial, OpenMP e CUDA utilizando o SLURM para alocar um nó no OGBON apenas enquanto estiver executando.

In [ ]:
%%writefile slurm.sh
#!/bin/sh

#SBATCH --job-name=OMP_CUDA                    # Job name
#SBATCH --nodes=10                             # Run all processes on 2 nodes
#SBATCH --partition=gpushortc                  # Partition OGBON
#SBATCH --output=out_%j.log                    # Standard output and error log
#SBATCH --ntasks-per-node=32                   # 1 job per node
#SBATCH --account=treinamento                  # Account of the group

PASS=$1

./bruteForce $PASS > seq

>omp
for i in 1 2 4 8 16 32 64 128 256 512 1024 2048
do
  OMP_NUM_THREADS=$i ./bruteForce-omp $PASS >> omp
done

./bruteForce-cuda $PASS > cuda

Overwriting slurm.sh


In [ ]:
!sbatch slurm.sh _Hacka1

Submitted batch job 553398


Esse script executa as soluções MPI e Híbrida separadamente, pois essas davam erro ao tentar utilizar o SLURM.

In [ ]:
%%writefile exec_mpi.sh
#!/bin/sh

PASS=$1

>mpi
>mpi+omp

for i in 1 2 4 8 16 32
do
  mpirun -x MXM_LOG_LEVEL=error -np $i ./bruteForce-mpi $PASS 2>/dev/null >> mpi
done

BEST_THREAD=$(awk 'BEGIN{best_time=-1; best_thread=""} {if(best_time == -1 || $3 <= best_time){best_time = $3; best_thread = $2;}} END{print best_thread}' omp)

for p in 2 4 8 16 32
do
  THREAD_SAMPLES=0
  for t in $((BEST_THREAD/32)) $((BEST_THREAD/16)) $((BEST_THREAD/8)) $((BEST_THREAD/4)) $((BEST_THREAD/2)) $((BEST_THREAD))
  do
    if [ $((p*t)) -lt 4096 ]; then
        OMP_NUM_THREADS=$t mpirun -x MXM_LOG_LEVEL=error -np $p ./bruteForce-mpi+omp $PASS 2>/dev/null >> mpi+omp
    fi
  done
done

Overwriting exec_mpi.sh


In [ ]:
!bash exec_mpi.sh _Hacka1

Esse script coleta todos os dados coletados nas últimas execuções e cria um gráfico de tempo de execução e outro de speedup para cada uma das técnicas de paralelização.

In [ ]:
%%bash
#!/bin/sh

export LC_NUMERIC="en_US.UTF-8"

set -eu pipefail

clear 

PASS=$1

###################################
# FUNCTIONS                       #
###################################

create_plot_script_time() {
cat <<EOF >time.plt
set title "Gráfico de Tempo de Execução" 
set ylabel "Tempo (Segundos)"
set xlabel "Threads/Processos"

set style line 1 lt 2 lc rgb "violet" lw 2 
set terminal png size 2000,720 enhanced
set output 'time.png'

set xtics nomirror
set ytics nomirror
set key top left
set key box
set style data lines


plot "file_comparison.data" using 3:xtic(2) title "$1" ls 1 with linespoints
EOF
}

create_plot_script_speedup() {
cat <<EOF >speedup.plt
set title "Gráfico de Speedup" 
set ylabel "Speedup"
set xlabel "Threads/Processos"

set style line 1 lt 2 lc rgb "violet" lw 2 
set terminal png size 2000,720 enhanced
set output 'speedup.png'

set xtics nomirror
set ytics nomirror
set key top left
set key box
set style data lines

plot "file_speedup.data" using 2:xtic(1) title "$1" ls 1 with linespoints
EOF
}

plot_graphs(){
DATA_FILE=$1    
    
echo "#####################"
echo $DATA_FILE
echo "#####################"

###################################
# Experimental Times              #
###################################

pr -m -t -s\  $DATA_FILE  | awk '{print $1,"\t",$2,"\t",$3}' >> file_comparison.data

echo " "
echo "    [#][SENHA]  [T/P]  [TIME]"
cat -n  file_comparison.data

#####################
# SPEEDUP           #
#####################

awk -v SEQ=$SEQUENTIAL_TIME '{print $2, " ",((SEQ*1000)/($3*1000))}' file_comparison.data >> fspeed0 

pr -m -t -s\  fspeed0  | awk '{print $1,"\t",$2}' >> file_speedup.data

echo " "
echo "    [#][T/P]    [SPEEDUP]"
cat -n file_speedup.data

#####################
# PLOTING           #
#####################

create_plot_script_time $DATA_FILE
create_plot_script_speedup $DATA_FILE
gnuplot "time.plt"
gnuplot "speedup.plt"

mv time.png  time_$DATA_FILE.png
mv speedup.png  speedup_$DATA_FILE.png
    
rm -f *.txt file* fspeed* *.data mm *.plt
echo " "
}

#####################
# MAIN              #
#####################

SEQUENTIAL_TIME=$(< seq)

for i in omp mpi mpi+omp
do
  plot_graphs $i
done

echo " "
echo "[END] " 
echo " "

#####################
omp
#####################
 
    [#][SENHA]  [T/P]  [TIME]
     1	_Hacka1 	 1 	 136.00
     2	_Hacka1 	 2 	 137.00
     3	_Hacka1 	 4 	 141.00
     4	_Hacka1 	 8 	 142.00
     5	_Hacka1 	 16 	 145.00
     6	_Hacka1 	 32 	 17.00
     7	_Hacka1 	 64 	 31.00
     8	_Hacka1 	 128 	 65.00
     9	_Hacka1 	 256 	 124.00
    10	_Hacka1 	 512 	 121.00
    11	_Hacka1 	 1024 	 105.00
    12	_Hacka1 	 2048 	 87.00
 
    [#][T/P]    [SPEEDUP]
     1	1 	 1.00735
     2	2 	 1
     3	4 	 0.971631
     4	8 	 0.964789
     5	16 	 0.944828
     6	32 	 8.05882
     7	64 	 4.41935
     8	128 	 2.10769
     9	256 	 1.10484
    10	512 	 1.13223
    11	1024 	 1.30476
    12	2048 	 1.57471
 
#####################
mpi
#####################
 
    [#][SENHA]  [T/P]  [TIME]
     1	_Hacka1 	 1 	 1315.00
     2	_Hacka1 	 2 	 1325.00
     3	_Hacka1 	 8 	 1418.00
     4	_Hacka1 	 16 	 1416.00
     5	_Hacka1 	 32 	 164.00
 
    [#][T/P]    [SPEEDUP]
     1	1 	 0.104183
     2	2 	 0.103396
     3	8 

Esse script apenas exibe os dados coletados das últimas execuções do código sequencial e CUDA. 

<h5>
  <center>
    <b>Figura 7</b> - Gráfico de tempo de execução da solução OMP
  </center>
</h5>
<img src="./time_omp.png" alt="Tempo de execução OMP">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

<h5>
  <center>
    <b>Figura 8</b> - Gráfico de Speedup da solução OMP
  </center>
</h5>
<img src="./speedup_omp.png" alt="Speedup OMP">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

Na solução de bruteforce com OMP, o speedup do script é o mesmo entre 1 e 16 threads, mas tem seu maior pico com o uso de 32 threads, a partir disso, diminui gradativamente, e de 256 threads em diante aumenta de forma progressiva, mas não se aproxima do pico. Isto se dá, pois as threads acabam roubando recurso umas das outras, além de que em um computador nada é tão simples que possa ser resolvido adicionando mais e mais threads, acrescentar mais do que o necessário pode atrapalhar mais do que ajudar. 

<h5>
  <center>
    <b>Figura 9</b> - Gráfico de Tempo de execução da solução MPI
  </center>
</h5>
<img src="./time_mpi.png" alt="Tempo de execução MPI">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

<h5>
  <center>
    <b>Figura 10</b> - Gráfico de Speedup da solução MPI
  </center>
</h5>
<img src="./speedup_mpi.png" alt="Speedup MPI">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

Na solução de bruteforce com a biblioteca MPI, não se atinge de fato um speedup significativo, todas as execuções demoram mais que o código sequencial e com o uso de 32 processos o tempo de execução se aproxima do sequencial, mas continua pior. 

O teste a partir de 64 processos resultou em erro e após diferentes experimentações a equipe concluiu que o uso do MPI não é a melhor solução para o problema apresentado.


<h5>
  <center>
    <b>Figura 11</b> - Gráfico do tempo de execução da solução MPI+OMP
  </center>
</h5>
<img src="./time_mpi+omp.png" alt="Tempo de execução MPI+OMP">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

<h5>
  <center>
    <b>Figura 12</b> - Gráfico Speedup da solução MPI+OMP
  </center>
</h5>
<img src="./speedup_mpi+omp.png" alt=" Speedup MPI+OMP">
<h5>
  <center>
    Fonte: Própria
  </center>
</h5>

Nesta decifração foi utilizado a junção das bibliotecas OpenMP e OpenMPI em uma solução híbrida, neste caso usando threads e processos para tentar acelerar o tempo de execução. O maior speedup desta estratégia foi com o uso de quatro processos e oito threads, em diante têm-se variações de tempo, mas nota-se a formação de um padrão, visto que a multiplicação entre threads e processos geram speedups bem parecidos.

In [ ]:
%%bash
echo "#####################"
echo "SEQUENTIAL TIME"
echo "#####################"

SEQUENTIAL_TIME=$(< seq)

echo " "
echo $SEQUENTIAL_TIME

echo " "
echo "#####################"
echo "CUDA"
echo "#####################"

echo " "
echo "[SENHA] [B/T]           [TIME]"
cat cuda

echo " "
echo "[B/T]         [SPEEDUP]"
awk -v SEQ=$SEQUENTIAL_TIME '{print $2, " ",((SEQ*1000)/($3*1000))}' cuda

#####################
SEQUENTIAL TIME
#####################
 
137
 
#####################
CUDA
#####################
 
[SENHA] [B/T]           [TIME]
_Hacka1	B2560/T1024	11.00
 
[B/T]         [SPEEDUP]
B2560/T1024   12.4545


A solução utilizando CUDA foi a mais eficiente tendo em vista os resultados acima. Seu speedup foi de 12.45x mais rápido, isso se dá porque realizar processos de tamanha magnitude em GPU é mais eficiente.

### Parâmetros Ótimos de Execução

1. OpenMP = 1 nó + 32 Threads
2. MPI = 1 nó + 32 Processos
3. MPI + OpenMP = 1 nó + 4 Processos + 8 Threads
4. CUDA = G1D B2560DT1024D (80 * 32, 1024)

### Tempo de execução em segundos das aplicações

Esse script testa cadeias de letras "z" em cada solução com seus parâmetros ótimos e armazena o tempo de execução e speedup. 

In [ ]:
%%writefile exec_seqz.sh
#!/bin/sh

BEST_THREAD=$(awk 'BEGIN{best_time=-1; best_thread=""} {if(best_time == -1 || $3 <= best_time){best_time = $3; best_thread = $2;}} END{print best_thread}' omp)
BEST_PROCESS=$(awk 'BEGIN{best_time=-1; best_process=""} {if(best_time == -1 || $3 <= best_time){best_time = $3; best_process = $2;}} END{print best_process}' mpi)

echo "Senha (Entradas) | Sequencial | OpenMP | MPI  | Híbrido | CUDA" >time_seqz
echo "Senha (Entradas) | OpenMP | MPI  | Híbrido | CUDA" >speedup_seqz

for PASS in zzzzzzz zzzzzzzz zzzzzzzzz zzzzzzzzzz
do
    LENGTH=${#PASS}

    ./bruteForce $PASS >tmp_seq
    SEQUENTIAL_TIME=$(<tmp_seq)
    
    OMP_NUM_THREADS=$BEST_THREAD ./bruteForce-omp $PASS >tmp_omp
    mpirun -x MXM_LOG_LEVEL=error -np $BEST_PROCESS ./bruteForce-mpi $PASS 2>/dev/null >tmp_mpi
    OMP_NUM_THREADS=8 mpirun -x MXM_LOG_LEVEL=error -np 4 ./bruteForce-mpi+omp $PASS 2>/dev/null >tmp_mpi+omp
    ./bruteForce-cuda $PASS >tmp_cuda
        

    pr -m -t -s\  tmp_omp  tmp_mpi  tmp_mpi+omp  tmp_cuda  | awk -v pwd=$PASS -v pwd_len=$LENGTH -v seq=$SEQUENTIAL_TIME '{printf  "(%dz) %s     |     %d      |   %s | %s |    %s | %s\n", pwd_len, pwd, seq, $3, $6, $9, $12;}' >>time_seqz
    pr -m -t -s\  tmp_omp  tmp_mpi  tmp_mpi+omp  tmp_cuda  | awk -v pwd=$PASS -v pwd_len=$LENGTH -v seq=$SEQUENTIAL_TIME '{printf  "(%dz) %s     |   %1.2f | %1.2f |    %1.2f | %1.2f\n", pwd_len, pwd, (((seq)*1000)/(($3)*1000)), (((seq)*1000)/(($6)*1000)), (((seq)*1000)/(($9)*1000)), (((seq)*1000)/(($12)*1000));}' >>speedup_seqz
done
        
rm -f ./tmp*

Writing exec_seqz.sh


In [ ]:
!bash exec_seqz.sh

In [ ]:
!cat time_seqz

Senha (Entradas) | Sequencial | OpenMP | MPI  | Híbrido | CUDA
(7z) zzzzzzz     |     3      |   1.00 | 1.00 |    1.00 | 1.00
(8z) zzzzzzzz     |     75      |   2.00 | 3.00 |    6.00 | 1.00
(9z) zzzzzzzzz     |     2000      |   72.00 | 72.00 |    144.00 | 5.00


### Speedup

In [ ]:
!cat speedup_seqz

Senha (Entradas) | OpenMP | MPI  | Híbrido | CUDA
(7z) zzzzzzz     |   3.00 | 3.00 |    3.00 | 3.00
(8z) zzzzzzzz     |   37.50 | 25.00 |    12.50 | 75.00
(9z) zzzzzzzzz     |   27.78 | 27.78 |    13.89 | 400.00


## Conclusões

Considerando o que foi pesquisado e assimilado em sala de aula, a equipe completou com sucesso o objetivo do Hackaton, foram feitas diferentes soluções para o algoritmo de bruteforce, e foi utilizado um shell script para coleta dos dados e criação dos gráficos e também foi usufrui de grande parte dos assuntos e bibliotecas aprendidos em classe.

As soluções foram testadas com diferentes caracteres e diferentes tamanhos que podem conter em uma senha. Seus respectivos resultados são exibidos em gráficos, discutidos e comparados para poder definir qual é a melhor solução. Considerando o maior speedup, a melhor solução para este problema foi com o uso de CUDA que foi 12,45x mais rápido que o sequencial.
 
Para realizar o código em cuda a equipe recorreu a diferentes conceitos, entre eles, stride loop, arquitetura de GPU e memória unificada. Essas funções foram necessárias para testar diferentes senhas de diferentes tamanhos e usar o máximo de recursos disponíveis em uma GPU.

Sendo assim, este Hackaton serviu como aprendizado e também como apanhado geral dos materiais estudados. Uma sugestão para testes futuros seria a aplicação deste algoritmo para tentar quebrar senhas de roteadores e switches em conjunto com o professor de redes.

## Referências Biliográficas

* G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

* S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

* David A. Patterson and John L. Hennessy. Computer Orga- nization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

* An Introduction to Parallel Programming by Peter S. Pache- co. Morgan Kauffman.

* W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabri- bridge U. Press, 1994.

* How to: Convert an OpenMP Loop that Uses Cancellation to Use the Concurrency Runtime. learn.microsoft.com, 2021. Disponível em: https://learn.microsoft.com/en-us/cpp/parallel/concrt/convert-an-openmp-loop-that-uses-cancellation?view=msvc-170. Acesso em: 9, nov 2022.

* CUDA Zone. developer.nvidea.com. Disponível em: https://developer.nvidia.com/cuda-zone. Acesso em: 19, nov 2022.

* gprof Command. ibm.com. Disponível em: https://www.ibm.com/docs/en/aix/7.1?topic=g-gprof-command. Acesso em: 20, nov 2022.

* OpenMP. openmp.org. Disponível em: https://www.openmp.org/. Acesso em: 20, nov 2022.

* Open MPI:Open Source High Performance Computing. open-mpi.org. Disponível em: https://www.open-mpi.org/. Acesso em: 20, nov 2022.

* The Difference Between Asynchronous And Multi-Threading. baeldung.com, 2022. Disponível em:https://www.baeldung.com/cs/async-vs-multi-threading. Acesso em: 20, nov 2022.

* CUDA – Streaming Multiprocessors. thebeardsage.com, 2020. Disponível em: http://thebeardsage.com/cuda-streaming-multiprocessors/#:~:text=Each%20architecture%20in%20GPU%20consists,several%20thread%20blocks%20in%20parallel.  Acesso em: 20, nov 2022. 

* ARAFA, Yehia. Low Overhead Instruction Latency Characterization for NVIDIA GPGPUs. researchgate.net, 2019. Disponível em: https://www.researchgate.net/figure/Typical-NVIDIA-GPU-architecture-The-number-of-SMXs-and-the-computation-resources-inside_fig1_333308943. Acesso em: 20, nov 2022.

* SINGAL, Dhruv. N Ways to SAXPY: Demonstrating the Breadth of GPU Programming Options. developer.nvidea.com, 2021. Disponível em: https://developer.nvidia.com/blog/n-ways-to-saxpy-demonstrating-the-breadth-of-gpu-programming-options/. Acesso em: 20, nov 2022.

* LIPPERT, Eric. É sempre garantido que uma aplicação com múltiplas threads rode mais rápido que usando uma única thread?. pt.stackoverflow.com, 2022. Disponível em: https://pt.stackoverflow.com/questions/1946/É-sempre-garantido-que-uma-aplicação-com-múltiplas-threads-rode-mais-rápido-que. Acesso em: 20, nov 2022.
